# Compiling the april and june data

#### Two sections -  1. Compiling exit/entry point angle data for learning and test flights and  2. Compile traj.nv (all z_coord for all frames of all bees)

## 1. Compiling exit/entry point angle data for learning and test flights

In [1]:
import os
import pandas as pd
import numpy as np
import math

# matplot
import matplotlib
import matplotlib.pyplot as plt

from matplotlib.projections import PolarAxes
from mpl_toolkits.axisartist.floating_axes import GridHelperCurveLinear, FloatingSubplot
import mpl_toolkits.axisartist.grid_finder as gf

import matplotlib.pyplot as plt
import seaborn as sns

# Config
import configparser
import plotly.graph_objects as go

from num2words import num2words 

# stat 
# import scipy.stats as stats
# import statsmodels.api as sm
# from statsmodels.formula.api import ols
# from statsmodels.stats.multicomp import pairwise_tukeyhsd

### Compile data 

In [94]:
# Empty df
compiled_data_june_april = pd.DataFrame(columns=['flight','number','month','beeID','recording',
                                                 'FirstHorzAngle','FirstVertAngle','SecondHorzAngle',
                                                 'SecondVertAngle','first_pt_height','second_pf_height',
                                                 'duration'])
compiled_data_june_april


,flight,number,month,beeID,recording,FirstHorzAngle,FirstVertAngle,SecondHorzAngle,SecondVertAngle,first_pt_height,second_pf_height,duration


In [95]:
# compile learning flights

idx = 0 # to append the data in df

for ii in ['april','june']: 
    
    # choosing the path 
    if ii == 'april': 
        common_path = '/path_for_april/'
    elif ii == 'june':
        common_path = '/path_for_june/'
    
    
    # sorted data
    lf_Reco_sorted = pd.read_csv('{}/lf_Reco_sorted.csv'.format(common_path))
    
    
    bee_id_col = lf_Reco_sorted.columns
    
    # loop over recording (1st, 2nd ....)
    for jj in range(len(lf_Reco_sorted)):
        
        # Which learning flight (1st,2nd,...)
        Nth_row_lf =  lf_Reco_sorted.iloc[jj]
        
        # loop over all beeIDs
        for kk in range(len(bee_id_col)):
            recording_lf = Nth_row_lf[kk]
            bee_id = bee_id_col[kk]
            
            # if NOT excluded recording (if NOT NaN)
            if pd.isna(recording_lf) == False: 
                
                # append available data
                compiled_data_june_april.at[idx,'flight'] = 'learn' # learning/test flight
                compiled_data_june_april.at[idx,'number'] = num2words(jj+1) # which set (1st, 2nd, 3rd ....) 
                
                compiled_data_june_april.at[idx,'month'] = ii # april/july
                
                compiled_data_june_april.at[idx,'beeID'] = bee_id 
                compiled_data_june_april.at[idx,'recording'] = recording_lf # recording
                
                # get exit/entry angle data
                # path to files
                lf_path = os.path.join(common_path,bee_id,'{}_learn'.format(recording_lf)) 
                
                # csv files
                exit_angle_data = pd.read_csv(os.path.join(lf_path,
                                                           'exitDistAng_data_{}_learn.csv'.format(recording_lf)))
                entry_angle_data = pd.read_csv(os.path.join(lf_path,
                                                            'entryDistAng_data_{}_learn.csv'.format(recording_lf)))
                
                # append df for angle                
                compiled_data_june_april.at[idx,'FirstHorzAngle'] = exit_angle_data['horz_angle'].iloc[0]
                compiled_data_june_april.at[idx,'FirstVertAngle'] = exit_angle_data['vert_angle'].iloc[0]
                compiled_data_june_april.at[idx,'SecondHorzAngle'] = entry_angle_data['horz_angle'].iloc[-1]
                compiled_data_june_april.at[idx,'SecondVertAngle'] = entry_angle_data['vert_angle'].iloc[-1]
                
                # append height data
                entry_meanHeight_data = pd.read_csv(os.path.join(lf_path,
                                                         "entrymid_mean_df_{}_learn.csv".format(recording_lf)),
                                  index_col=0)
                exit_meanHeight_data = pd.read_csv(os.path.join(lf_path,
                                                         "exitmid_mean_df_{}_learn.csv".format(recording_lf)),
                                  index_col=0)
                
                compiled_data_june_april.at[idx,'first_pt_height'] = exit_meanHeight_data['z_mean'].iloc[0]
                compiled_data_june_april.at[idx,'second_pf_height'] = entry_meanHeight_data['z_mean'].iloc[-1]

                
                traj_nv_lf = pd.read_csv(os.path.join(lf_path,'traj_nv_{}_learn.csv'.format(recording_lf)))
                
                compiled_data_june_april.at[idx,'duration'] = traj_nv_lf['timeseries'].iloc[-1]-traj_nv_lf['timeseries'].iloc[0]
                idx += 1  # increament idx          
                

In [96]:
# compile test flights - same code as above for test flights
# DO NOT REDEFINE 'idx'

for ii in ['april','june']:
    if ii == 'april':
        common_path = '/path_for_april/'
    elif ii == 'june':
        common_path = 'path_for_june'
    
    
    tf_Reco_sorted = pd.read_csv('{}/tf_Reco_sorted.csv'.format(common_path))
    
    bee_id_col = tf_Reco_sorted.columns
    
    for jj in range(len(tf_Reco_sorted)):
        Nth_row_tf = tf_Reco_sorted.iloc[jj]
        
        for kk in range(len(bee_id_col)):
            recording_tf = Nth_row_tf[kk]
            bee_id = bee_id_col[kk]
            
            if pd.isna(recording_tf) == False:
                
                compiled_data_june_april.at[idx,'flight'] = 'test'
                compiled_data_june_april.at[idx,'number'] = num2words(jj+1)
                compiled_data_june_april.at[idx,'month'] = ii
                compiled_data_june_april.at[idx,'beeID'] = bee_id
                compiled_data_june_april.at[idx,'recording'] = recording_tf
                
                tf_path = '{0}/{1}/{2}_test'.format(common_path,bee_id,recording_tf)
                entry_angle_data = pd.read_csv('{0}/entryDistAng_data_{1}_test.csv'.format(tf_path,recording_tf))
                
                compiled_data_june_april.at[idx,'FirstHorzAngle'] = entry_angle_data['horz_angle'].iloc[0]
                compiled_data_june_april.at[idx,'FirstVertAngle'] = entry_angle_data['vert_angle'].iloc[0]
                compiled_data_june_april.at[idx,'SecondHorzAngle'] = entry_angle_data['horz_angle'].iloc[-1]
                compiled_data_june_april.at[idx,'SecondVertAngle'] = entry_angle_data['vert_angle'].iloc[-1]
                
                
                entry_meanHeight_data = pd.read_csv(os.path.join(tf_path,
                                                         "entrymid_mean_df_{}_test.csv".format(recording_tf)))
                
                compiled_data_june_april.at[idx,'first_pt_height'] = entry_meanHeight_data['z_mean'].iloc[0]
                compiled_data_june_april.at[idx,'second_pf_height'] = entry_meanHeight_data['z_mean'].iloc[-1]
                
                # append df for angle   
                traj_nv_tf = pd.read_csv(os.path.join(tf_path,'traj_nv_{}_test.csv'.format(recording_tf)))
                compiled_data_june_april.at[idx,'duration'] = traj_nv_tf['timeseries'].iloc[-1]-traj_nv_tf['timeseries'].iloc[0]
                # add mean of horz and vert angle ()
                
                
                idx += 1
        

In [117]:
# setting index 
compiled_data_june_april = compiled_data_june_april.set_index(['flight','number'])
compiled_data_june_april

month beeID        recording FirstHorzAngle FirstVertAngle  \
flight number                                                               
learn  one     april    1R  20210416_161052     189.529786      70.187692   
       one     april   28W  20210414_155821     237.353278      67.479735   
       one     april    2R  20210414_163606     254.429968      65.307252   
       one     april   31G  20210414_141848     142.525424       58.34731   
       one     april   33R  20210414_123241      245.29373       48.80975   
...              ...   ...              ...            ...            ...   
test   five     june   36W  20210622_144644     211.466045      39.104936   
       five     june   55R  20210625_133023     155.362295      43.649901   
       five     june   71W  20210623_141928     173.670258      23.875244   
       five     june   73R  20210624_151338      97.497087       2.362677   
       five     june    8Y  20210622_151137     228.881463       4.757031   

              SecondHorzAngle SecondVertAngle first_pt_height  \
flight number                                                   
learn  one         120.107747       19.155056         0.28763   
       one         210.747701       30.494517        0.306324   
       one          97.140099       49.139568        0.298301   
       one          146.29535       37.215456        0.273044   
       one         199.761385       41.095057        0.286427   
...                       ...             ...             ...   
test   five         205.81007       23.578354        0.276177   
       five        185.605444        7.923228        0.269357   
       five        173.455312        1.386533        0.217011   
       five        210.960068       32.296919        0.015708   
       five        131.045959        4.562128        0.037011   

              second_pf_height  duration  
flight number                             
learn  one            0.148915  1665.872  
       one            0.257781   595.728  
       one            0.284391   467.328  
       one             0.26261  1327.072  
       one            0.286308   2572.64  
...                        ...       ...  
test   five           0.209836   133.904  
       five           0.067649     170.0  
       five           0.011912    223.04  
       five           0.262932    73.424  
       five           0.036934    39.584  

[135 rows x 10 columns]

In [8]:
# path tocompiled data of april and july
compiled_data_path = '/compiled_data_path/'


In [119]:
# compiled_data_june_april.to_csv('{}/compiled_data_june_april.csv'.format(compiled_data_path))
compiled_data_june_april.to_csv(os.path.join(compiled_data_path,'compiled_data_june_april.csv'))

## 2. Compile traj.nv (all z_coord for all frames of all bees)

In [2]:
# expty data frame
compiled_z_coord = pd.DataFrame()
# compiled_z_coord = pd.DataFrame()


In [3]:
common_path = '/common_path/'

In [4]:
# Getting z-coordinate data for learning flight

for month in ['april','june']: 
    
    if month =="april":
        month_data_path = os.path.join(common_path,"{}/data/msc_project".format("exp_20210413"))
    elif month =="june":
        month_data_path = os.path.join(common_path,"{}/data/msc_project".format("exp_20210614"))

    lf_Reco_sorted = pd.read_csv(os.path.join(month_data_path,"lf_Reco_sorted.csv"))

    for ii in range(len(lf_Reco_sorted)): 

        bee_id_col = lf_Reco_sorted.columns

        Nth_row_reco_lf = lf_Reco_sorted.iloc[ii].dropna() # list of recording 
        beeId_reco_index = Nth_row_reco_lf.index

        set_num = num2words(ii+1) # need later

        for bee_id in beeId_reco_index:
            lf_path = os.path.join(month_data_path,bee_id,'{}_learn'.format(Nth_row_reco_lf.loc[bee_id]))

            traj_nv = pd.read_csv(os.path.join(lf_path,"traj_nv_{}_learn.csv".format(Nth_row_reco_lf.loc[bee_id])),
                                  index_col=0)



            df_z_coord = pd.DataFrame({"flight":"learn",
                                       "set":set_num,
                                       "month":month,
                                       "beeID":bee_id,
                                       "recording":Nth_row_reco_lf.loc[bee_id],
                                       "z_coord":traj_nv.z})

            compiled_z_coord = pd.concat([compiled_z_coord,df_z_coord])



In [5]:
# Getting z-coordinate data for test flight

for month in ['april','june']: 
    
    if month =="april":
        month_data_path = os.path.join(common_path,"{}/data/msc_project".format("exp_20210413"))
    elif month =="june":
        month_data_path = os.path.join(common_path,"{}/data/msc_project".format("exp_20210614"))

    tf_Reco_sorted = pd.read_csv(os.path.join(month_data_path,"tf_Reco_sorted.csv"))

    for ii in range(len(tf_Reco_sorted)): 

        bee_id_col = tf_Reco_sorted.columns

        Nth_row_reco_tf = tf_Reco_sorted.iloc[ii].dropna() # list of recording 
        beeId_reco_index = Nth_row_reco_tf.index

        set_num = num2words(ii+1) # need later

        for bee_id in beeId_reco_index:
            tf_path = os.path.join(month_data_path,bee_id,'{}_test'.format(Nth_row_reco_tf.loc[bee_id]))

            traj_nv = pd.read_csv(os.path.join(tf_path,"traj_nv_{}_test.csv".format(Nth_row_reco_tf.loc[bee_id])),
                                  index_col=0)



            df_z_coord = pd.DataFrame({"flight":"test",
                                       "set":set_num,
                                       "month":month,
                                       "beeID":bee_id,
                                       "recording":Nth_row_reco_tf.loc[bee_id],
                                       "z_coord":traj_nv.z})

            compiled_z_coord = pd.concat([compiled_z_coord,df_z_coord])



In [6]:
compiled_z_coord = compiled_z_coord.set_index(["flight","set","month"])
compiled_z_coord

beeID        recording   z_coord
flight set  month                                 
learn  one  april    1R  20210416_161052 -0.046918
            april    1R  20210416_161052 -0.047324
            april    1R  20210416_161052 -0.046930
            april    1R  20210416_161052 -0.046049
            april    1R  20210416_161052 -0.045177
...                 ...              ...       ...
test   five june     8Y  20210622_151137 -0.029756
            june     8Y  20210622_151137 -0.031053
            june     8Y  20210622_151137 -0.031568
            june     8Y  20210622_151137 -0.021767
            june     8Y  20210622_151137 -0.028454

[4350745 rows x 3 columns]

In [9]:
# save the flight
compiled_z_coord.to_csv(os.path.join(compiled_data_path,'compiled_trajnv_heightData.csv'))